In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 20657825
paper_name = 'fabrizio_longo_2010' 

In [46]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [47]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [27]:
original_data = pd.read_excel('raw_data/journal.pgen.1001024.s006.xlsx', sheet_name='SuppleTable2_v2.txt', skiprows=2)

In [28]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4130 x 17


In [29]:
original_data.head()

,ORF,GENE,RMSE,Correlation,yko1d9/yko1d3,yko1d11/yko1d3,yko1d15/yko1d3,yko1d20/yko1d3,yko2d9/yko2d3,yko2d11/yko2d3,yko2d15/yko2d3,yko2d20/yko2d3,Avg(d9/d3),Avg(d11/d3),Avg(d15/d3),Avg(d20/d3),Used for Clustering
0,YER107C,GLE2,2.164961,0.895598,0.232,0.349,1.939,3.201,-0.450,2.509,2.842,6.779,-0.1090,1.4290,2.3905,4.9900,0
1,YDR463W,STP1,3.098199,0.578208,0.407,0.479,3.732,1.522,0.350,4.112,5.383,6.262,0.3785,2.2955,4.5575,3.8920,0
2,YMR275C,BUL1,0.476648,0.984977,2.028,2.659,3.375,3.580,1.431,2.222,2.806,3.386,1.7295,2.4405,3.0905,3.4830,1
3,YDR443C,SSN2,0.967582,0.869372,-0.472,0.157,2.811,3.494,-0.078,2.018,2.500,3.322,-0.2750,1.0875,2.6555,3.4080,1
4,YOR061W,CKA2,0.284371,0.927151,0.943,1.242,1.686,2.299,0.594,1.550,2.005,2.370,0.7685,1.3960,1.8455,2.3345,1


In [30]:
original_data['orf'] = original_data['ORF'].astype(str)

In [31]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [32]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [33]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, GENE, RMSE, Correlation, yko1d9/yko1d3, yko1d11/yko1d3, yko1d15/yko1d3, yko1d20/yko1d3, yko2d9/yko2d3, yko2d11/yko2d3, yko2d15/yko2d3, yko2d20/yko2d3, Avg(d9/d3), Avg(d11/d3), Avg(d15/d3), Avg(d20/d3), Used for Clustering, orf]
Index: []


In [34]:
original_data.set_index('orf', inplace=True)

In [35]:
data_cols = [c for c in original_data.columns if 'yko' in c]
data_cols

['yko1d9/yko1d3',
 'yko1d11/yko1d3',
 'yko1d15/yko1d3',
 'yko1d20/yko1d3',
 'yko2d9/yko2d3',
 'yko2d11/yko2d3',
 'yko2d15/yko2d3',
 'yko2d20/yko2d3']

In [36]:
original_data = original_data[data_cols].copy()

In [37]:
original_data = original_data.apply(pd.to_numeric, axis=1, errors='coerce')

In [38]:
original_data = original_data.groupby(original_data.index).mean()

In [39]:
original_data.shape

(4105, 8)

In [40]:
original_data.columns = [4711, 4781, 4782, 4783, 4711, 4781, 4782, 4783]

In [41]:
original_data = original_data.T
original_data = original_data.groupby(original_data.index).mean()
original_data = original_data.T

In [42]:
original_data.shape

(4105, 4)

In [43]:
original_data.head()

,4711,4781,4782,4783
orf,,,,
YAL002W,-1.3615,-1.3550,-1.1645,-0.9160
YAL004W,-0.1650,-0.1945,-0.6020,-0.2100
YAL005C,0.0850,-0.1490,0.4745,0.6225
YAL007C,0.2400,-0.0855,-0.0790,-0.0265
YAL008W,0.2355,-0.1405,-0.5285,-0.2160


# Prepare the final dataset

In [48]:
data = original_data.copy()

In [49]:
dataset_ids = [4711, 4781, 4782, 4783]
datasets = datasets.reindex(index=dataset_ids)

In [50]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [51]:
data.head()

dataset_id,4711,4781,4782,4783
data_type,value,value,value,value
orf,,,,
YAL002W,-1.3615,-1.3550,-1.1645,-0.9160
YAL004W,-0.1650,-0.1945,-0.6020,-0.2100
YAL005C,0.0850,-0.1490,0.4745,0.6225
YAL007C,0.2400,-0.0855,-0.0790,-0.0265
YAL008W,0.2355,-0.1405,-0.5285,-0.2160


## Subset to the genes currently in SGD

In [52]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [53]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,4711,4781,4782,4783
,data_type,value,value,value,value
gene_id,orf,,,,
2,YAL002W,-1.3615,-1.3550,-1.1645,-0.9160
1863,YAL004W,-0.1650,-0.1945,-0.6020,-0.2100
4,YAL005C,0.0850,-0.1490,0.4745,0.6225
5,YAL007C,0.2400,-0.0855,-0.0790,-0.0265
6,YAL008W,0.2355,-0.1405,-0.5285,-0.2160


# Normalize

In [54]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [55]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [56]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,4711,4781,4782,4783,4711,4781,4782,4783
,data_type,value,value,value,value,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,
2,YAL002W,-1.3615,-1.3550,-1.1645,-0.9160,-2.838911,-2.448770,-1.607520,-1.207497
1863,YAL004W,-0.1650,-0.1945,-0.6020,-0.2100,-0.447994,-0.436453,-0.864466,-0.274049
4,YAL005C,0.0850,-0.1490,0.4745,0.6225,0.051571,-0.357556,0.557576,0.826653
5,YAL007C,0.2400,-0.0855,-0.0790,-0.0265,0.361301,-0.247446,-0.173590,-0.031431
6,YAL008W,0.2355,-0.1405,-0.5285,-0.2160,0.352309,-0.342817,-0.767373,-0.281982


# Print out

In [57]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [58]:
from IO.save_data_to_db3 import *

In [59]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/4 [00:00<?, ?it/s]

Deleting all datasets for PMID 20657825...
Inserting the new data...


100%|██████████| 4/4 [00:26<00:00,  6.53s/it]

Updating the data_modified_on field...
